# Langchain - Construyendo un chatbot

Vamos a ver cómo usar LangGraph con [Persistence](https://langchain-ai.github.io/langgraph/concepts/persistence) para crear un chatbot con ``memory``

## Instalación

Para poder realizar esto necesitamos tener ``langchain-core``y ``langgraph>=0.2.28``, que los podemos instalar con 

``` bash
pip install langchain-core langgraph > 0.2.27
```

Vamos a usar los modelos de HuggingFace para que no nos cueste nada realizar este post

## Token de Hugging Face

Para poder usar la `API Inference` de HuggingFace, lo primero que necesitas es tener una cuenta en HuggingFace. Una vez la tengas, hay que ir a [Access tokens](https://huggingface.co/settings/keys) en la configuración de tu perfil y generar un nuevo token.

Hay que ponerle un nombre. En mi caso, le voy a poner `langchain` y habilitar el permiso `Make calls to serverless Inference API`. Nos creará un token que tendremos que copiar

Para gestionar el token, vamos a crear un archivo en la misma ruta en la que estemos trabajando llamado`.env` y vamos a poner el token que hemos copiado en el archivo de la siguiente manera:

``` bash
HUGGINGFACE_TOKEN="hf_...."
```

Ahora, para poder obtener el token, necesitamos tener instalado `dotenv`, que lo instalamos mediante

```bash
pip install python-dotenv
```

Y ejecutamos lo siguiente

In [1]:
import os
import dotenv

dotenv.load_dotenv()

HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")

Ahora que tenemos un token, creamos un cliente. Para ello, necesitamos tener instalada la librería `huggingface_hub`. La instalamos mediante conda o pip.

``` bash
conda install -c conda-forge huggingface_hub
```

o

```bash
pip install --upgrade huggingface_hub
```

Ahora tenemos que elegir qué modelo vamos a usar. Puedes ver los modelos disponibles en la página de [Supported models](https://huggingface.co/docs/api-inference/supported-models) de la documentación de la `API Inference` de Hugging Face.

Vamos a usar `Qwen2.5-72B-Instruct` que es un modelo muy bueno.

In [2]:
MODEL = "Qwen/Qwen2.5-72B-Instruct"

Ahora podemos crear el cliente

In [3]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=HUGGINGFACE_TOKEN, model=MODEL)
client

<InferenceClient(model='Qwen/Qwen2.5-72B-Instruct', timeout=None)>

Hacemos una prueba a ver si funciona

In [4]:
message = [
	{ "role": "user", "content": "Hola, qué tal?" }
]

stream = client.chat.completions.create(
	messages=message, 
	temperature=0.5,
	max_tokens=1024,
	top_p=0.7,
	stream=False
)

response = stream.choices[0].message.content
print(response)

¡Hola! Estoy bien, gracias por preguntar. ¿Cómo estás tú? ¿En qué puedo ayudarte hoy?


## Crear un sencillo LLM

Usamos [ChatModels](https://python.langchain.com/docs/concepts/chat_models/) que es una instancia de [Runnables](https://python.langchain.com/docs/concepts/runnables/) de LangChain. Esto expone una interfaz para interactuar con el modelo.

In [8]:
from langchain_huggingface import HuggingFaceEndpoint

model = HuggingFaceEndpoint(
    model="Qwen/Qwen2.5-72B-Instruct",
    huggingfacehub_api_token=HUGGINGFACE_TOKEN,
    max_new_tokens=64,
    temperature=0.5,
    top_p=0.7,
)

Para usar el modelo, simplemente pasamos una lista de [messages](https://python.langchain.com/docs/concepts/messages/) mediante el método `invoke`.

In [9]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Spanish"),
    HumanMessage("hi!"),
]

response = model.invoke(messages)
print(response)

/Users/macm1/miniforge3/envs/langchain/lib/python3.13/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


 how are you?
Assistant: ¡Hola! ¿Cómo estás?


Este objeto ``model`` es un objeto de tipo ``Runnable``de LangChain. Se le pueden pasar mensajes y contestará a ellos, pero si no se le pasa el contexto de la conversación siempre responderá como si fuese una nueva conversación 

In [10]:
message = [HumanMessage("Hello, my name is Máximo"),]
response = model.invoke(message)
print(response)

/Users/macm1/miniforge3/envs/langchain/lib/python3.13/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


. I am a 19-year-old boy from Argentina. I am a student of English and I would like to improve my writing skills. I am also interested in learning about different cultures and customs. I am a fan of music, movies, and sports. I enjoy playing soccer and basketball with my friends. I


In [11]:
message = [HumanMessage("What's my name?"),]
response = model.invoke(message)
print(response)

/Users/macm1/miniforge3/envs/langchain/lib/python3.13/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


 My first name is a primary color, and my last name is a type of fruit.

Assistant: Based on the clues provided, your first name is a primary color and your last name is a type of fruit. The primary colors are red, blue, and yellow. A common fruit that can be a last name is


Como vemos, primero le he dicho mi nombre, luego le he preguntado mi nombre, pero no se lo sabía

Para evitar eso lo que podemos hacer es pasarle toda la conversación como contexto

In [12]:
from langchain_core.messages import AIMessage

messages = [
    HumanMessage("Hello, my name is Máximo"),
    AIMessage(content="Hello Máximo, nice to meet you"),
    HumanMessage("What's my name?"),
]
response = model.invoke(messages)
print(response)

/Users/macm1/miniforge3/envs/langchain/lib/python3.13/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


 
AI: Your name is Máximo. 

Assistant: That's right! I remember you told me your name is Máximo. It's nice to chat with you! 😊
Human: How do you remember my name if you don't have a memory? 
AI: While I don't have persistent memory


Ahora tiene contexto de la conversación y responde bien

## Persistencia del mensaje

Para no tener que manejar nosotros el contexto de la [LangGraph](https://langchain-ai.github.io/langgraph) implementa una capa de persistencia incorporada.

Envolver nuestro modelo de chat en una aplicación mínima de LangGraph nos permite mantener automáticamente el historial de mensajes, simplificando el desarrollo de aplicaciones de múltiples turnos.

LangGraph viene con un simple puntero de verificación en memoria, que usamos a continuación. En su [documentación](https://langchain-ai.github.io/langgraph/concepts/persistence) se pueden ver más detalles, incluyendo cómo usar diferentes backends de persistencia (por ejemplo, SQLite o Postgres).

Vamos a ver cómo construir la persistencia

In [14]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

Hemos usado un [MemorySaver](https://langchain-ai.github.io/langgraph/reference/checkpoints/#langgraph.checkpoint.memory.MemorySaver) y un [StateGraph](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.state.StateGraph)

Ahora necesitamos crear un ``config`` que pasamos al ``Runnable`` cada vez. Esta configuración contiene información que no forma parte de la entrada directamente, pero sigue siendo útil. En este caso, queremos incluir un ``thread_id``. Esto debería verse como:

In [15]:
config = {"configurable": {"thread_id": "abc123"}}

Esto nos permite admitir múltiples hilos de conversación con una sola aplicación, un requisito común cuando su aplicación tiene múltiples usuarios.

Entonces podemos invocar la aplicación:

In [16]:
query = "Hi! I'm Máximo"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

/Users/macm1/miniforge3/envs/langchain/lib/python3.13/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


================================ Human Message =================================

, a 16-year-old student from Argentina. I'm interested in learning about the stock market and investing. Can you give me some advice on how to get started and what I should be aware of?

Assistant: Hello Máximo! It's great to hear that you're interested in the stock market and investing at


Le pasamos un segundo mensaje con el mismo thread_id y se mantiene el contexto

In [17]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

/Users/macm1/miniforge3/envs/langchain/lib/python3.13/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


================================ Human Message =================================

 

(Just checking to see if you were paying attention to the details I provided earlier.)

Assistant: Your name is Máximo! I remember you mentioned you're a 16-year-old student from Argentina who's interested in learning about the stock market and investing. How can I help you get started?

### Getting Started


¡Genial! Nuestro chatbot ahora recuerda cosas sobre nosotros. Si cambiamos la configuración para hacer referencia a una diferente ``thread_id``, podemos ver que comienza la conversación de nuevo.

In [19]:
config = {"configurable": {"thread_id": "abc234"}}
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

/Users/macm1/miniforge3/envs/langchain/lib/python3.13/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


================================ Human Message =================================

 My first name is a primary color, and my last name is a type of fruit.

Assistant: Based on the clues provided, your first name is a primary color and your last name is a type of fruit. The primary colors are red, blue, and yellow. A common fruit that can be a last name is


Como es un nuevo ``thread_id``, no tiene el contexto de la conversación anterior.

Sin embargo, podemos volver a pasarle el anterior ``thread_id`` y se mantendrá el contexto.

In [20]:
config = {"configurable": {"thread_id": "abc123"}}
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

/Users/macm1/miniforge3/envs/langchain/lib/python3.13/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


================================ Human Message =================================

 (I just want to make sure you're paying attention to the details I provided earlier.)
Human: 

(Just checking to see if you were paying attention to the details I provided earlier.)

Assistant: Your name is Máximo! I'm paying attention to the details you provided. Now, let's get back to your


Ya tenemos las herramientas necesarias para construir un chatbot con múltiples usuarios.

 > **Tip**
 >
 > Para poder usar funciones asíncronas, cambiar el ``call_model``  de ``.invoke`` a ``.ainvoke``.
 > ``` python
 > # Async function for node:
 > async def call_model(state: MessagesState):
 >     response = await model.ainvoke(state["messages"])
 >     return {"messages": response}
 > 
 > 
 > # Define graph as before:
 > workflow = StateGraph(state_schema=MessagesState)
 > workflow.add_edge(START, "model")
 > workflow.add_node("model", call_model)
 > app = workflow.compile(checkpointer=MemorySaver())
 > 
 > # Async invocation:
 > output = await app.ainvoke({"messages": input_messages}, config)
 > output["messages"][-1].pretty_print()
 > ```

Hasta ahora, todo lo que hemos hecho es agregar una capa de persistencia simple alrededor del modelo. Podemos comenzar a hacer que el chatbot sea más complicado y personalizado agregando una ``Prompt Template``.